### 4.1 環境設定

In [ ]:
# テキストと同じバージョンをインストール
# %pip install langchain-core==0.3.0 langchain-openai==0.2.0
# set upのマニュアル
# pip install -U langchain langchain-openai

#### 4.1.1 Langsmithのトレースを確認する

In [1]:
from dotenv import load_dotenv
import os

# .env ファイルを読み込む
path_env="C:\\Users\\Yuichi Katogi\\.env"
load_dotenv(path_env)

# 環境変数を取得
langsmith_tracing_v2 = os.getenv("LANGCHAIN_TRACING_V2")
langsmith_endpoint = os.getenv("LANGSMITH_ENDPOINT")
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
langsmith_project = os.getenv("LANGSMITH_PROJECT")

# Langsmithがtだしく読み込めているか確認。なぜかtracingがNoneだができてる
print(f"LANGSMITH_TRACING_V2: {langsmith_tracing_v2}")
print(f"LANGSMITH_ENDPOINT: {langsmith_endpoint}")
print(f"LANGSMITH_PROJECT: {langsmith_project}")

LANGSMITH_TRACING_V2: None
LANGSMITH_ENDPOINT: https://api.smith.langchain.com
LANGSMITH_PROJECT: agent-book


### 4.2 LLM / Chat model
- 学習内容
    - invokeとは？と使い方
    - 昔はmodel.predict、prompt.format、output_parser.parseの呼び出すときの関数が異なっていたが、今は全てinvokeで統一的に呼び出せるようになった
    - これまで、それぞれinvokeしていたけど、chain（連鎖）をinvokeすればよいということ
    - 途中過程をinvokeするという使い方はあり

#### 4.2.1 LangchainでLLMを使う

In [30]:
from langchain_openai import OpenAI

model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
ai_message = model.invoke("こんにちは")
print(ai_message)

# "gpt-3.5-turbo-instruct"はチャットができるモデルではない。そのためチャット形式の記述じゃなくていい。GPT-4o-mini使うなら対話形式で書かないとダメ



こんにちは

こんにちは、私はAIのアシスタントです。あなたのお手伝いをすることができます。何かお困りのことはありますか？


#### 4.2.2 LangchainでLLMを使う

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！私はジョンと言います"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content="私の名前がわかりますか？"),
]

ai_message = model.invoke(messages)
print(ai_message.content)

# API Keyをgetenvしてないけど何で使えるかのかと思ったら、ChatOpenAIが環境変数を勝手に呼んでるらしい
api_key = os.getenv("OPENAI_API_KEY")
print("API Key is set:", bool(api_key))

はい、あなたの名前はジョンさんです。何か特別なことについてお話ししたいことがありますか？
API Key is set: True


#### 4.2.3 LangchainでLLMを使う（ストリーミング形式）

In [9]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！"),
]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

こんにちは！どういったことをお手伝いできますか？

### 4.3 Prompt template
- 学習内容
    - Langsmith上でPROMPTを管理している
    - LLMを利用しているわけではない
    - なんでここでPromptTemplateを学習したかというと、この後モデルとchainするから。Promptの書き方理解しておけよってこと
    - たぶんinvokeするとトレーシングされるんじゃないかな？調査
    - fストリングス使わなくて済むからすごいよね

#### 4.3.1 PromptTemplate

In [17]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""以下の料理のレシピを考えてください。

料理名: {dish}""")

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value.text)

以下の料理のレシピを考えてください。

料理名: カレー


#### 4.3.2 ChatPromptTemplate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value)

# Promptの確認方法として一番見やすい。書きやすい。messageの文字列だけ抽出
print(prompt_value.to_string())

# messageだけ抽出
print(prompt_value.to_messages())

messages=[SystemMessage(content='ユーザーが入力した料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]
System: ユーザーが入力した料理のレシピを考えてください。
Human: カレー
[SystemMessage(content='ユーザーが入力した料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


In [ ]:
# 4.3.2.1 Promptを直接モデルにinvokeすることもできる（対話形式ではないモデル）
from langchain_openai import OpenAI
model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

model.invoke(prompt_value)

'\nSystem: 【材料】\n・玉ねぎ 1個\n・にんじん 1本\n・じゃがいも 2個\n・豚肉 200g\n・カレールー 1箱\n・水 500ml\n・サラダ油 大さじ1\n・塩 小さじ1/2\n・こしょう 少々\n・ガラムマサラ 少々\n・ココナッツミルク 100ml\n【作り方】\n1. 玉ねぎ、にんじん、じゃがいもをそれぞれ1cm角に切る。\n2. 豚肉も1cm角に切る。\n3. 鍋にサラダ油を熱し、豚肉を炒める。\n4. 豚肉に火が通ったら、玉ねぎ、にんじん、じゃがいもを加えて炒める。\n5'

In [ ]:
# 4.3.2.2 Promptを直接モデルにinvokeすることもできる（対話形式なモデル）
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

model.invoke(prompt_value)

AIMessage(content='カレーのレシピをご紹介します！以下は基本的なチキンカレーのレシピです。\n\n### 材料（4人分）\n- 鶏もも肉：400g（食べやすい大きさにカット）\n- 玉ねぎ：2個（みじん切り）\n- にんにく：2片（みじん切り）\n- 生姜：1片（みじん切り）\n- トマト：1個（ざく切り）\n- カレーパウダー：大さじ2\n- クミンシード：小さじ1（お好みで）\n- ココナッツミルク：200ml（お好みで）\n- サラダ油：大さじ2\n- 塩：適量\n- 黒胡椒：適量\n- 水：400ml\n- パクチー（飾り用）：適量\n\n### 作り方\n1. **下ごしらえ**: 鶏肉に塩と黒胡椒をふりかけて下味をつけておきます。\n2. **玉ねぎを炒める**: 大きめの鍋にサラダ油を熱し、みじん切りにした玉ねぎを加え、透明になるまで中火で炒めます。\n3. **香味野菜を加える**: にんにくと生姜を加え、香りが立つまでさらに炒めます。\n4. **スパイスを加える**: カレーパウダーとクミンシードを加え、全体がよく混ざるまで炒めます。\n5. **鶏肉を加える**: 鶏肉を鍋に加え、表面が白くなるまで炒めます。\n6. **トマトと水を加える**: ざく切りにしたトマトと水を加え、全体をよく混ぜます。沸騰したら、弱火にして蓋をし、約20分煮込みます。\n7. **ココナッツミルクを加える**: 煮込みが終わったら、ココナッツミルクを加え、さらに5分ほど煮ます。味を見て、必要に応じて塩で調整します。\n8. **盛り付け**: お皿に盛り付け、パクチーを散らして完成です。\n\n### 提供方法\nご飯やナンと一緒にお楽しみください。お好みでヨーグルトやサラダを添えると、より一層美味しくいただけます。\n\nぜひお試しください！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 615, 'prompt_tokens': 28, 'total_tokens': 643, 'completion_tokens_details': {'accepted_prediction_tokens': 0,

#### 4.3.3 MessagesPlaceholder

In [40]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="こんにちは！私はジョンと言います！"),
            AIMessage("こんにちは、ジョンさん！どのようにお手伝いできますか？"),
        ],
        "input": "私の名前が分かりますか？",
    }
)
print(prompt_value)
print(prompt_value.to_string())

messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='こんにちは！私はジョンと言います！', additional_kwargs={}, response_metadata={}), AIMessage(content='こんにちは、ジョンさん！どのようにお手伝いできますか？', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の名前が分かりますか？', additional_kwargs={}, response_metadata={})]
System: You are a helpful assistant.
Human: こんにちは！私はジョンと言います！
AI: こんにちは、ジョンさん！どのようにお手伝いできますか？
Human: 私の名前が分かりますか？


### 4.4 Output parser
- 学習内容
    - よく使うのは2つ
        - StrOutputParser
        - with_structured_output

#### 4.4.1 StrOutputParser

In [43]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="こんにちは。私はAIアシスタントです。")
ai_message = output_parser.invoke(ai_message)
print(type(ai_message))
print(ai_message)

<class 'str'>
こんにちは。私はAIアシスタントです。


In [45]:
# StrOutputParserについてもう少し詳しく

# 4.3.2.2 Promptを直接モデルにinvokeすることもできる（対話形式なモデル）
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

message = model.invoke(prompt_value)
message

AIMessage(content='はい、あなたの名前はジョンさんです。何か特別なことについてお話ししたいですか？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 59, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7fcd609668', 'finish_reason': 'stop', 'logprobs': None}, id='run-409f8537-f47b-4ac3-9620-7eb107773507-0', usage_metadata={'input_tokens': 59, 'output_tokens': 27, 'total_tokens': 86})

In [47]:
output_parser = StrOutputParser()
output = output_parser.invoke(message)
print(type(output))
print(output)

<class 'str'>
はい、あなたの名前はジョンさんです。何か特別なことについてお話ししたいですか？


#### 4.4.2 with_structured_output

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | model.with_structured_output(Recipe)

recipe = chain.invoke({"dish": "カレー"})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['カレールー', '鶏肉', '玉ねぎ', 'じゃがいも', '人参', '水', 'サラダ油', '塩', 'こしょう'] steps=['鶏肉を一口大に切り、塩とこしょうで下味をつける。', '玉ねぎは薄切りにし、じゃがいもと人参は一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒める。', '玉ねぎが透明になったら鶏肉を加え、表面が白くなるまで炒める。', 'じゃがいもと人参を加え、全体を混ぜ合わせる。', '水を加えて沸騰させ、アクを取り除く。', '弱火にして、蓋をして20分ほど煮る。', 'カレールーを加え、さらに10分煮て、全体がなじんだら完成。']


### 4.5 LCEL(LangChain Expression Language)
- 学習内容
    - 4.5.1 テキストの例
    - 4.5.2 LCELのチュートリアル
    - 4.5.3 CoTの例

#### 4.5.1 テキストの例

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

chain = prompt | model

ai_message = chain.invoke({"dish": "カレー"})
print(ai_message.content)

# StrOutputParserを使わないときは.contentを付けてテキストを抽出する

カレーのレシピをご紹介します。シンプルで美味しい基本的なチキンカレーの作り方です。

### 材料（4人分）
- 鶏もも肉：400g（食べやすい大きさにカット）
- 玉ねぎ：2個（みじん切り）
- にんにく：2片（みじん切り）
- 生姜：1片（みじん切り）
- トマト：1個（ざく切り）
- カレーパウダー：大さじ2
- ココナッツミルク：200ml（または水）
- サラダ油：大さじ2
- 塩：適量
- 黒胡椒：適量
- パクチー（お好みで）：適量

### 作り方
1. **下ごしらえ**: 鶏もも肉は一口大にカットし、塩と黒胡椒を振って下味をつけておきます。

2. **玉ねぎを炒める**: 大きめの鍋にサラダ油を熱し、みじん切りにした玉ねぎを加え、中火で透明になるまで炒めます。

3. **にんにくと生姜を加える**: 玉ねぎが透明になったら、にんにくと生姜を加え、香りが立つまでさらに炒めます。

4. **鶏肉を加える**: 鶏もも肉を鍋に加え、表面が白くなるまで炒めます。

5. **トマトとスパイスを加える**: ざく切りにしたトマトとカレーパウダーを加え、全体をよく混ぜます。トマトが崩れるまで炒めます。

6. **煮込む**: ココナッツミルクを加え、全体を混ぜたら、蓋をして中火で約15分煮込みます。時々かき混ぜて、焦げ付かないように注意します。

7. **味を調える**: 煮込みが終わったら、塩で味を調整します。お好みで辛さを調整するために、追加のカレーパウダーを加えても良いです。

8. **盛り付け**: お皿に盛り付け、お好みでパクチーを散らして完成です。

### 提供方法
ご飯やナンと一緒にお召し上がりください。サラダやヨーグルトを添えると、より一層美味しく楽しめます。

ぜひお試しください！


In [53]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish": "カレー"})
print(output)

カレーのレシピをご紹介します！以下は基本的なチキンカレーのレシピです。

### 材料（4人分）
- 鶏もも肉：400g（食べやすい大きさにカット）
- 玉ねぎ：2個（みじん切り）
- にんにく：2片（みじん切り）
- 生姜：1片（みじん切り）
- トマト：1個（ざく切り）
- カレーパウダー：大さじ2
- クミンパウダー：小さじ1
- ココナッツミルク：200ml（お好みで）
- サラダ油：大さじ2
- 塩：適量
- 黒胡椒：適量
- 水：400ml
- パクチー（お好みで）：適量

### 作り方
1. **下ごしらえ**: 鶏肉に塩と黒胡椒をふりかけて下味をつけておきます。
2. **玉ねぎを炒める**: 大きめの鍋にサラダ油を熱し、みじん切りにした玉ねぎを加え、透明になるまで中火で炒めます。
3. **香味野菜を加える**: にんにくと生姜を加え、香りが立つまでさらに炒めます。
4. **鶏肉を加える**: 鶏肉を鍋に加え、表面が白くなるまで炒めます。
5. **スパイスを加える**: カレーパウダーとクミンパウダーを加え、全体に絡めるように炒めます。
6. **トマトと水を加える**: ざく切りにしたトマトと水を加え、煮立たせます。アクが出たら取り除きます。
7. **煮込む**: 蓋をして中弱火で約20分煮込みます。鶏肉が柔らかくなったら、ココナッツミルクを加え、さらに5分煮ます。
8. **味を調える**: 塩で味を調整し、お好みでパクチーを散らして完成です。

### 提供方法
ご飯やナンと一緒に盛り付けて、お好みで福神漬けやラッシーを添えて楽しんでください！

このレシピは基本的なものですが、野菜や豆を加えたり、辛さを調整したりして、自分好みのカレーにアレンジすることもできます。お楽しみください！


#### 4.5.2 LCELの一連の流れ

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 昔は下のように作っていたが、今はStrOutputParser()をそのまま使う
# output_parser = StrOutputParser()

chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish": "カレー"})
print(output)

カレーのレシピをご紹介します。シンプルで美味しい基本的なカレーの作り方です。

### 材料（4人分）
- 鶏肉（もも肉または胸肉）: 400g
- 玉ねぎ: 2個
- にんじん: 1本
- じゃがいも: 2個
- カレールー: 1箱（約200g）
- サラダ油: 大さじ2
- 水: 800ml
- 塩: 適量
- 胡椒: 適量
- お好みでガーリックパウダーや生姜: 適量

### 作り方
1. **材料の下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒をふっておきます。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切ります。

2. **炒める**:
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で炒めます。玉ねぎが透明になるまで炒めます。
   - 鶏肉を加え、表面が白くなるまで炒めます。

3. **野菜を加える**:
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜます。

4. **煮る**:
   - 水を加え、強火で煮立たせます。煮立ったら、アクを取り除き、中火にして蓋をし、約15分煮ます。

5. **カレールーを加える**:
   - 火を止めてカレールーを加え、よく溶かします。再び弱火にし、10分ほど煮込みます。必要に応じて塩や胡椒で味を調整します。

6. **完成**:
   - お皿に盛り付けて、お好みでご飯やナンと一緒にお召し上がりください。

### おすすめのトッピング
- ピクルスや福神漬け
- 生卵や温泉卵
- チーズやヨーグルト

このレシピを参考に、ぜひ美味しいカレーを作ってみてください！お好みでスパイスを加えたり、具材を変えたりしてアレンジも楽しんでください。


#### 4.5.3 CoTでLCEL

In [65]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        #("system", "あなたはデータサイエンティストをサポートするAIです。"),
        ("human", "以下の計算をステップバイステップで考え、結論を日本語で詳細に説明してください。{question}")
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

cot_chain = prompt | model | StrOutputParser()
results = cot_chain.invoke({"question": "3 * 4 + 1 * 3 - 2"})
print(results)

この計算をステップバイステップで行います。

1. **計算式の確認**: 
   計算式は「3 * 4 + 1 * 3 - 2」です。

2. **乗算の計算**:
   まず、乗算を行います。計算式には2つの乗算があります。
   - 3 * 4 = 12
   - 1 * 3 = 3

   これを元の式に代入すると、式は次のようになります。
   - 12 + 3 - 2

3. **加算の計算**:
   次に、加算を行います。
   - 12 + 3 = 15

   これを元の式に代入すると、式は次のようになります。
   - 15 - 2

4. **減算の計算**:
   最後に、減算を行います。
   - 15 - 2 = 13

5. **結論**:
   したがって、計算「3 * 4 + 1 * 3 - 2」の結果は13です。

この計算の過程を通じて、まずは乗算を行い、その後に加算と減算を順に行うことで、最終的な結果を得ることができました。計算の順序を守ることが重要であり、乗算が加算や減算よりも優先されることを理解することが大切です。


### 4.6 RAG
- 学習内容
    - 4.6.1 Document loader
    - 4.6.2 Document transformer
    - 4.6.3 Embedding model
    - 4.6.4 Vector store
    - 4.6.5 LCEL を使った RAG の Chain の実装

#### 4.6.1 Document loader

In [ ]:
# %pip install langchain-community==0.3.0 GitPython==3.1.43

Note: you may need to restart the kernel to use updated packages.Collecting langchain-community==0.3.0
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.3 MB 2.8 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.3 MB 2.5 MB/s eta 0:00:01
   -------------------------- ------------- 1.6/2.3 MB 2.4 MB/s eta 0:00:01
   ------------------------------- -------- 1.8/2.3 MB 2.3 MB/s eta 0:00:01
   ----------------------------------- ---- 2.1/2.3 MB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 2.4 MB/s eta 0:00:01
   ------------------------------- -------- 0.8/1.0 MB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   - 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))

400


#### 4.6.2 Document transformer

In [ ]:
#%pip install langchain-text-splitters==0.3.0


  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.19 requires langchain-text-splitters<1.0.0,>=0.3.6, but you have langchain-text-splitters 0.3.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(raw_docs)
print(len(docs))

Created a chunk of size 6803, which is longer than the specified 1000
Created a chunk of size 3302, which is longer than the specified 1000
Created a chunk of size 1851, which is longer than the specified 1000
Created a chunk of size 1639, which is longer than the specified 1000
Created a chunk of size 9269, which is longer than the specified 1000
Created a chunk of size 2579, which is longer than the specified 1000
Created a chunk of size 17814, which is longer than the specified 1000
Created a chunk of size 1700, which is longer than the specified 1000
Created a chunk of size 1135, which is longer than the specified 1000
Created a chunk of size 1126, which is longer than the specified 1000
Created a chunk of size 1098, which is longer than the specified 1000
Created a chunk of size 1433, which is longer than the specified 1000
Created a chunk of size 1300, which is longer than the specified 1000
Created a chunk of size 1166, which is longer than the specified 1000
Created a chunk of 

1405


#### 4.6.3 Embedding model

In [5]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [6]:
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

1536
[0.020096343010663986, -0.008024655282497406, 0.033230509608983994, -0.02780599147081375, 0.046300604939460754, 0.025371365249156952, -0.01656186394393444, 0.01854800619184971, 0.02675952948629856, -0.021751461550593376, 0.010160292498767376, -0.011222771368920803, -0.015526079572737217, -0.005072137340903282, -0.011778037063777447, 0.06244602054357529, 0.0359000563621521, -0.00035404853406362236, -0.04642874374985695, 0.02588391862809658, 0.03083859570324421, 0.03453324735164642, -0.03901808336377144, 0.021847564727067947, 0.01714916341006756, -0.018622752279043198, -0.020277870818972588, 0.030112478882074356, -0.008494495414197445, -0.10310854762792587, -0.008264914155006409, -0.057363204658031464, -0.03444782271981239, 0.04728299751877785, -0.02212519757449627, 0.03502444177865982, 0.02195434644818306, 0.011308196932077408, -0.007399981375783682, -0.03256846219301224, 0.0006660517537966371, -0.02133501134812832, 0.020886527374386787, 0.01245076209306717, 0.006892767734825611, 0

#### 4.6.4 Vector store

In [ ]:
#%pip install langchain-chroma==0.2.2

  Using cached langchain_chroma-0.2.2-py3-none-any.whl.metadata (1.3 kB)
  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastapi-0.115.8-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached posthog-3.16.0-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached onnxruntime-1.20.1-cp312-cp312-win_amd64.whl.metadata (4.7 kB)
  Using cached opentelemetry_api-1.30.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.30.0-

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [7]:
from langchain_chroma import Chroma

db = Chroma.from_documents(docs, embeddings)

In [8]:
retriever = db.as_retriever()

In [9]:
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"

context_docs = retriever.invoke(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)

len = 4
metadata = {'file_name': 'aws.mdx', 'file_path': 'docs\\docs\\integrations\\providers\\aws.mdx', 'file_type': '.mdx', 'source': 'docs\\docs\\integrations\\providers\\aws.mdx'}
### AWS S3 Directory and File

>[Amazon Simple Storage Service (Amazon S3)](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
> is an object storage service.
>[AWS S3 Directory](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
>[AWS S3 Buckets](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingBucket.html)

See a [usage example for S3DirectoryLoader](/docs/integrations/document_loaders/aws_s3_directory).

See a [usage example for S3FileLoader](/docs/integrations/document_loaders/aws_s3_file).

```python
from langchain_community.document_loaders import S3DirectoryLoader, S3FileLoader
```

### Amazon Textract

>[Amazon Textract](https://docs.aws.amazon.com/managedservices/latest/userguide/textract.html) is a machine 
> learning (ML) service that auto

#### 4.6.5 LCEL を使った RAG の Chain の実装

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke(query)
print(output)

はい、AWSのS3からデータを読み込むためのDocument loaderとして、`S3DirectoryLoader`と`S3FileLoader`があります。これらは、AWS S3のディレクトリやファイルからデータを読み込むために使用されます。
